In [6]:
from tqdm.auto import tqdm
import numba as numba
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [8]:
# Define the combined model
def create_model():
    inputs = layers.Input(shape=(28, 28, 1))
    x = layers.Conv2D(32, 3, activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Reshape((-1, 64))(x)  # Reshape for the self-attention layer
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Create and compile the model
model = create_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 reshape_1 (Reshape)         (None, 25, 64)            0   

In [9]:
 # Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255  

In [10]:
# Train the model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(
    x_train, 
    y_train, 
    epochs=120,
    use_multiprocessing=True,
    batch_size=250,
    validation_split=0.3,
    callbacks=[early_stopping]
)

Epoch 1/120
168/168 [==============================] - 18s 107ms/step - loss: 0.4343 - accuracy: 0.8736 - val_loss: 0.1278 - val_accuracy: 0.9637
Epoch 2/120
168/168 [==============================] - 18s 108ms/step - loss: 0.1005 - accuracy: 0.9693 - val_loss: 0.0859 - val_accuracy: 0.9738
Epoch 3/120
168/168 [==============================] - 18s 107ms/step - loss: 0.0683 - accuracy: 0.9795 - val_loss: 0.0682 - val_accuracy: 0.9800
Epoch 4/120
168/168 [==============================] - 18s 107ms/step - loss: 0.0525 - accuracy: 0.9838 - val_loss: 0.0587 - val_accuracy: 0.9822
Epoch 5/120
168/168 [==============================] - 17s 102ms/step - loss: 0.0446 - accuracy: 0.9864 - val_loss: 0.0605 - val_accuracy: 0.9819
Epoch 6/120
168/168 [==============================] - 20s 122ms/step - loss: 0.0383 - accuracy: 0.9884 - val_loss: 0.0570 - val_accuracy: 0.9831
Epoch 7/120
168/168 [==============================] - 17s 99ms/step - loss: 0.0334 - accuracy: 0.9902 - val_loss: 0.0502 - 

In [12]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 6s - loss: 0.0446 - accuracy: 0.9885 - 6s/epoch - 19ms/step

Test accuracy: 0.9884999990463257
